In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from qap_lp.models import *
from qap_lp.conf import *

# instance_name = 'esc16h' ill conditions
instance_name = 'bur26a'

kwargs = {}
msk_params = {**MSK_DEFAULT, **kwargs}
qap_params = {**QAP_DEFAULT, **kwargs}

#
qap_params['scaling'] = 'l1'
qap_params['mu'] = 0.1
# coefficients
A0, B0 = parse(f'{QAP_INSTANCE}/{instance_name}.dat')

# parse known solution
_, best_obj, arr = parse_sol(f'{QAP_SOL}/{instance_name}.sln')

param = QAPParam(A0, B0, best_obj, arr, **qap_params)

# unpacking
A, B, n, m, e, E, ab = \
    param.A, param.B, param.n, param.m, param.e, param.E, param.ab
# more
nabla = QAPDerivativeL2Penalty(param, qap_params['mu'])
x = x0 = np.ones((n, n)) / n
nabla.obj(param.xo), nabla.obj(x)

(13.42935979608503, 17.187692098294942)

In [3]:
u, ga, v = np.linalg.svd(param.ab)

#### gradient projection

- projection may be computed by mosek
- or some other methods

#### project point onto $D_n$

- projection may be computed by mosek
- or some other methods

# eval

In [ ]:
x_int, _ = extract_sol_rounding(x, param.A, param.B)

In [ ]:
def check_obj_val(x_sol):
    _obj = A0.T.dot(x_sol).dot(B0).dot(x_sol.T).trace()
    print(f'original obj {_obj}')
    return _obj

In [ ]:
check_obj_val(x)

In [ ]:
nabla.original_obj(xol), nabla.obj(xol, mu)